In [ ]:
import HEAD_RECON
import multi_head
import copy
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.ndimage.morphology import binary_fill_holes
import sys
np.set_printoptions(threshold=sys.maxsize)
from scipy.signal import detrend
import matplotlib.gridspec as gridspec
%load_ext autoreload
%autoreload 2

In [ ]:
for i in range(1,16):
    print(i)
    head = HEAD_RECON.threeD_head.read_from_file(Sequence,i)
    head.full_filter()
    head.save()

In [17]:
# gen pickled heads
Sequence=1

In [18]:
# load heads
all_heads=[HEAD_RECON.threeD_head.load_from_pickle(Sequence,i) for i in range (1,16)]

In [20]:
# make head from first to last
def make_mhead(all_heads, first, last):
    all_heads[first].reset_positions()
    all_heads[first].reset_colors()
    all_heads[first+1].reset_positions()
    all_heads[first+1].reset_colors()   
    heads = multi_head.MultiHead.joined_heads(all_heads[first], all_heads[first+1])
    heads.links.append(multi_head.Link(left=all_heads[first+1].frame_id,right=all_heads[first].frame_id))
    for i in range(first+2,last+1):
        heads.links.append(multi_head.Link(left=all_heads[i].frame_id,right=all_heads[i-1].frame_id))
        all_heads[i].reset_positions()
        all_heads[i].reset_colors()
        heads.append(all_heads[i])        
        if i==last:
             heads.links.append(multi_head.Link(left=all_heads[first].frame_id,right=all_heads[i].frame_id))
    print ("completed")
    return heads
mhead = make_mhead(all_heads, 0, 14)
mhead.calc_keypoints()

completed


In [22]:
def calc_connections(mhead):
    for link in mhead.links:
        link.reset()
        mhead.ransac_from_link(link)        
calc_connections(mhead)

In [115]:
only_first_n=14
mhead.reset_all_head_positions()
link_index,err=mhead.get_next_unpositioned_link()
positioned_head_count=0
joined_heads=set() 
while(not link_index is None) and (positioned_head_count<only_first_n or only_first_n==-1):
    mhead.links[link_index].print_short()
    joined_heads.add(mhead.links[link_index].left)
    joined_heads.add(mhead.links[link_index].right)
    foo= mhead.transform_from_link(mhead.links[link_index])
    link_index,err=mhead.get_next_unpositioned_link()
    positioned_head_count=max(positioned_head_count+1,2)
joined_heads=sorted(joined_heads)
print(joined_heads)
no_heads= len(joined_heads)
joined_heads=[str(h) for h in joined_heads]
name= f"Seq_{mhead.heads[0].sequence_id}_frames_{'_'.join(joined_heads)}"
mhead.create_spheres(min(2/max(no_heads,0.2),1),name=name)

any_head_positioned False
5-4, Count=137, Err=0.0031
any_head_positioned True
6-5, Count=113, Err=0.0045
any_head_positioned True
7-6, Count=90, Err=0.0069
any_head_positioned True
4-3, Count=107, Err=0.0071
any_head_positioned True
3-2, Count=147, Err=0.0067
any_head_positioned True
8-7, Count=64, Err=0.0080
any_head_positioned True
2-1, Count=101, Err=0.0098
any_head_positioned True
1-15, Count=154, Err=0.0076
any_head_positioned True
15-14, Count=70, Err=0.0126
any_head_positioned True
14-13, Count=52, Err=0.0077
any_head_positioned True
13-12, Count=76, Err=0.0095
any_head_positioned True
12-11, Count=58, Err=0.0047
any_head_positioned True
11-10, Count=25, Err=0.0189
any_head_positioned True
[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15]


In [145]:
import pandas as pd
df=pd.DataFrame(columns=['Right', 'Left',  'Matches', 'Inliers','Variance'])
for link in mhead.links:
    df = df.append({'Right' : int(link.right) , 'Left' : int(link.left) ,  'Variance' : link.err, 'Inliers' : sum(link.inliers), 'Matches' : len(link.matches)} , ignore_index=True)
df[['Right', 'Left',  'Matches', 'Inliers']] = df[['Right', 'Left',  'Matches', 'Inliers']].astype(int)
print(df.to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &  Right &  Left &  Matches &  Inliers &  Variance \\
\midrule
0  &      1 &     2 &      254 &      101 &  0.009788 \\
1  &      2 &     3 &      300 &      147 &  0.006730 \\
2  &      3 &     4 &      262 &      107 &  0.007085 \\
3  &      4 &     5 &      246 &      137 &  0.003069 \\
4  &      5 &     6 &      239 &      113 &  0.004493 \\
5  &      6 &     7 &      181 &       90 &  0.006948 \\
6  &      7 &     8 &      138 &       64 &  0.007980 \\
7  &      8 &     9 &      104 &       18 &  0.027298 \\
8  &      9 &    10 &      116 &       27 &  0.018986 \\
9  &     10 &    11 &      104 &       25 &  0.018863 \\
10 &     11 &    12 &      137 &       58 &  0.004724 \\
11 &     12 &    13 &      193 &       76 &  0.009457 \\
12 &     13 &    14 &      168 &       52 &  0.007739 \\
13 &     14 &    15 &      198 &       70 &  0.012636 \\
14 &     15 &     1 &      283 &      154 &  0.007570 \\
\bottomrule
\end{tabular}



0.009787951290177649

In [45]:
mhead.links[0].left

2

In [99]:
mhead.create_mesh(f"{Sequence}_mesh")


completed


In [ ]:
nrows,ncols=3,2
screenwidth=12
plt.figure(figsize = (screenwidth,3/4*nrows*screenwidth/ncols))
gs1 = gridspec.GridSpec(nrows, ncols)
gs1.update(wspace=0.1, hspace=0.1)
for row in range(nrows):
    for col in range(ncols):
        idx=ncols*row+col
        ax=plt.subplot(gs1[idx])
        mhead.heads[idx].background_color=np.array([0,0,1])
        img= mhead.heads[idx].get_filtered_image()
        ax.imshow(img)
        ax.set_axis_off()        
        ax.text(x=10, y=463 , s=f"S_{Sequence}:{idx+1}/15",horizontalalignment='left',
                              verticalalignment='bottom',  bbox=dict(facecolor='white', alpha=1))        
        del ax
plt.savefig(f"2d_filtered_seq_{Sequence}")
plt.show()

In [ ]:
sequence_ids=[0,1,2]
head_ids=[0,5,7]
abcd="abcd"
nrows,ncols=3,4
screenwidth=12
plt.figure(figsize = (screenwidth,3/4*nrows*screenwidth/ncols))
gs1 = gridspec.GridSpec(nrows, ncols)
gs1.update(wspace=0.1, hspace=0.1)
for row in range(nrows):
    head = sequence_ids[row]
    head = sequence_ids[row]
    sequence= sequence_ids[row]
    head_idx=head_ids[row]
    for col in range(ncols):
#         print (sequence, head_idx, col)
        head=mheads[sequence].heads[head_idx]
        if col==0:
            img= head.twoD_image            
            
        elif col==1:
            head.reset_filters()
            head.filter_depth(1)
            head.background_color=np.array([0,0,1])
            img= head.get_filtered_image()
        elif col==2:
            head.remove_dangling()
            img= head.get_filtered_image()
        elif col==3:
            head.remove_background_color()
            head.background_color=np.array([0,0,1])
            img= head.get_filtered_image()
        idx=ncols*row+col
        ax=plt.subplot(gs1[idx])
        ax.imshow(img)
        ax.set_axis_off()        
        if col==0:
            ax.text(x=20, y=20 , s=f"S_{sequence+1}:{head_idx+1}/15",horizontalalignment='left',
                                  verticalalignment='top',  bbox=dict(facecolor='white', alpha=1))        

        ax.text(x=20, y=453 , s=f"{abcd[col]}",horizontalalignment='left',
                              verticalalignment='bottom',  bbox=dict(facecolor='white', alpha=1))        
        
        del ax
        
plt.savefig(f"2d_full_seq_{Sequence}")
plt.show()

In [ ]:
linkindex=6
mhead.reset_all_head_positions()
foo= mhead.transform_from_link(mhead.links[linkindex])
mhead.links[linkindex].print_short()
mhead.create_spheres(0.4)

In [100]:
import glob

In [109]:
glob.glob("images\Seq_1_frames_SIFT*.png")

['images\\Seq_1_frames_SIFT_10_11.png',
 'images\\Seq_1_frames_SIFT_11_12.png',
 'images\\Seq_1_frames_SIFT_12_13.png',
 'images\\Seq_1_frames_SIFT_13_14.png',
 'images\\Seq_1_frames_SIFT_14_15.png',
 'images\\Seq_1_frames_SIFT_15_1.png',
 'images\\Seq_1_frames_SIFT_1_2.png',
 'images\\Seq_1_frames_SIFT_2_3.png',
 'images\\Seq_1_frames_SIFT_3_4.png',
 'images\\Seq_1_frames_SIFT_4_5.png',
 'images\\Seq_1_frames_SIFT_5_6.png',
 'images\\Seq_1_frames_SIFT_6_7.png',
 'images\\Seq_1_frames_SIFT_7_8.png',
 'images\\Seq_1_frames_SIFT_9_10.png']